<a href="https://colab.research.google.com/github/BonneyBoja/Bonney-s-Portfolio/blob/main/GenerativeAI%2CRAG%2CAgenticAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain-community


In [ ]:
## Import Libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline

In [ ]:
# Load PDF
loader = PyPDFLoader("SOFTWARE DEVELOPMENT PROCESS MODELS.pdf")
docs = loader.load()


In [ ]:
#loading the pdf
from google.colab import files

print("Please upload the PDF file")
uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}"')


Please upload the PDF file


Saving SOFTWARE DEVELOPMENT PROCESS MODELS.pdf to SOFTWARE DEVELOPMENT PROCESS MODELS (2).pdf
User uploaded file "SOFTWARE DEVELOPMENT PROCESS MODELS (2).pdf"


In [ ]:
#Split Documents into Chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [ ]:
# Create Embeddings and Vector Store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# FAISS stores embeddings and allows fast similarity search
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# The pipeline wraps tokenizer + model for easy text generation
flan_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

def query_rag(question):
      # Step 1: Retrieve similar chunks from vector store
    relevant_docs = retriever.invoke(question)
     # Step 2: Combine chunk texts into single context string
    context = "\n".join([doc.page_content for doc in relevant_docs])
    # Step 3: Build prompt that restricts the model to ONLY use context
    prompt = f"Answer the question using only the context:\n\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"

  # Step 4: Generate response using FLAN-T5
    response = flan_pipeline(
        prompt,
        max_new_tokens=200,
        temperature=0.9,      # Creativity control (lower = deterministic, higher = more diverse)
        top_k=50,             # Only sample from the top-k most likely tokens
        top_p=0.9,            # Nucleus sampling: only sample from tokens with cumulative prob <= top_p
        do_sample=True        # Enables sampling (required for temperature/top-k/top-p to work)
    )
        # Step 5: Return text only
    return response[0]['generated_text']

print(query_rag("Summarize the key points of this document in a paragraph of 200 words."))

Device set to use cpu


Spiral development is a process in which a software product is developed in stages. Each stage is followed by a POC (Proof of Concept) to get customer feedback. Then in the subsequent spirals with higher clarity on requirements and design details a working model of the software is produced with a version number. These builds are sent to the customer for feedback.
